# Initializing your environment

## Setting up a virtual environment
(Optional but recommended)

```bash
virtualenv -p python3 venv
source venv/bin/activate
```
(Use `deactivate` to exit from `source` once you are done)

Alternatively you can also prefix all your `python` and `pip` commands with `./venv/bin/` (e.g: `./venv/bin/pip3 install -U pip`)



## Setting up Jupyter

In order to follow along on your computer:

```bash
pip3 install notebook
jupyter-notebook
```

## Installation of PyMISP

#### Make sure the submodules are up-to-date and cloned

```bash
git submodule update --init --recursive PyMISP/
```

#### Install PyMISP with the developer options

```bash
cd PyMISP
pip3 install -e .
```

#### To be able to use the additional PyMISP helpers

```bash
# Make sure the package required for pydeep is installed
sudo apt-get install -y libfuzzy-dev

pip3 install python-magic, lief, git+https://github.com/kbandla/pydeep.git
```

# Using the PyMISP objects

PyMISP is the python library used to deal with MISP format so you do not have to deal with the JSON blob yourself without knowing about the required and optional fields.

## MISPEvent

MISPEvent is the main class to use when you want to create/update events on a MISP instance.

In [1]:
from pymisp import MISPEvent

event = MISPEvent()

event.info = 'A fancy MISP Event'  # Required
event.distribution = 0  # Optional, sets the distribution level to "Your Organisation only"
event.threat_level_id = 2  # Optional, sets the threat level to "Medium"
event.analysis = 1  # Optional, set the analysis to "Ongoing"

print(event.to_json(indent=4))

{
    "uuid": "a646652d-10ec-4d31-be98-792d441d77de",
    "info": "A fancy MISP Event",
    "distribution": "0",
    "threat_level_id": "2",
    "analysis": "1"
}


### Set the event date

The date can be in many different formats. PyMISP normalizes it in a way that will be understood by your MISP instance: a date in the `YYYY-MM-DD` format.

In [2]:
event.set_date('2022-01-06')
print(f'From a text date: {event.date}')

from datetime import date
d = date.today()
event.set_date(d)
print(f'From a datetime.date date: {event.date}')

from datetime import datetime
d = datetime.now()
event.set_date(d)
# MISP expects a day, so the Hour will be dropped
print(f'From a datetime.datetime date: {event.date}')

From a text date: 2022-01-06
From a datetime.date date: 2025-01-22
From a datetime.datetime date: 2025-01-22 13:55:37.488421


### Tag an Event

An easy way to tag an Event

In [3]:
event.add_tag('tlp:white')

print(event.to_json(indent=4))

{
    "uuid": "a646652d-10ec-4d31-be98-792d441d77de",
    "Tag": [
        {
            "name": "tlp:white"
        }
    ],
    "info": "A fancy MISP Event",
    "distribution": "0",
    "threat_level_id": "2",
    "analysis": "1",
    "date": "2025-01-22"
}


## MISP Attribute

MISP Attributes are the raw pieces of data that can either be indicators of compromise (IoCs) or observed data.

They are defined with a triplet of required values being a `type`, a `category` and a `value`, and a bunch of optional fields.

The Attributes parameters are the following:
- **type** (required)
- **value** (required)
- **category**: the type of information (if not set, the default one for the given `type` is used)
- **to_ids**: defines whether the attribute defines some malicious data that should be blocked, or not (if not set, the default value for the given `type` is used)
- **distribution**: defaults to inherit from parent (event or object)
- **disable_correlation**: if you want to avoid correlations between events on that specific value
- **data**: for `malware-sample` and `attachment` types, BytesIO object of the file.

### A minimal and quick way of adding an attribute

In [4]:
attribute1 = event.add_attribute('ip-dst', '8.8.8.8')

print(attribute1.to_json(indent=4))

{
    "uuid": "e45dd98c-7506-4c9e-a22a-4a47c033d2d1",
    "type": "ip-dst",
    "value": "8.8.8.8",
    "category": "Network activity",
    "to_ids": true,
    "disable_correlation": false
}


### Set inline parameters

In [5]:
attribute2 = event.add_attribute('ip-dst', '8.8.8.9', to_ids=False, disable_correlation=True)

print(attribute2.to_json(indent=4))

{
    "uuid": "24315449-b8ef-48ec-a4fa-30d7eb5a4dde",
    "type": "ip-dst",
    "value": "8.8.8.9",
    "category": "Network activity",
    "to_ids": false,
    "disable_correlation": true
}


### Modify an existing attribute

Every parameter can be modified in a pythonic way

In [6]:
attribute1.to_ids = False

print(attribute1.to_json(indent=4))

{
    "uuid": "e45dd98c-7506-4c9e-a22a-4a47c033d2d1",
    "type": "ip-dst",
    "value": "8.8.8.8",
    "category": "Network activity",
    "to_ids": false,
    "disable_correlation": false
}


### Soft delete an attribute

The default approach on MISP is to soft delete data, which means it will not be displayed in the dafult view on MISP.  
The reason for doing this is to allow to push delete updates to instances we synchronise with.

In [7]:
attribute2.delete()

print(event.to_json(indent=4))

{
    "uuid": "a646652d-10ec-4d31-be98-792d441d77de",
    "Attribute": [
        {
            "uuid": "e45dd98c-7506-4c9e-a22a-4a47c033d2d1",
            "type": "ip-dst",
            "value": "8.8.8.8",
            "category": "Network activity",
            "to_ids": false,
            "disable_correlation": false
        },
        {
            "uuid": "24315449-b8ef-48ec-a4fa-30d7eb5a4dde",
            "type": "ip-dst",
            "value": "8.8.8.9",
            "category": "Network activity",
            "to_ids": false,
            "disable_correlation": true,
            "deleted": true
        }
    ],
    "Tag": [
        {
            "name": "tlp:white"
        }
    ],
    "info": "A fancy MISP Event",
    "distribution": "0",
    "threat_level_id": "2",
    "analysis": "1",
    "date": "2025-01-22"
}


### A more advanced way of passing the different parameters at once

In [8]:
from uuid import uuid4

attribute_uuid = uuid4()
print(attribute_uuid)

kwargs = {
    'to_ids': False,
    'disable_correlation': True,
    'category': 'Network activity',
    'uuid': attribute_uuid
}
attribute = event.add_attribute('ip-src', '1.1.1.1', **kwargs)

print(attribute.to_json(indent=4))

30a67450-adfd-4754-86ab-eab73150dee4
{
    "uuid": "30a67450-adfd-4754-86ab-eab73150dee4",
    "type": "ip-src",
    "value": "1.1.1.1",
    "category": "Network activity",
    "to_ids": false,
    "disable_correlation": true
}


### Using the MISPAttribute class

Allows you to play with the attribute before adding it to the event.

It is then possible to load the attribute from a JSON or from a dict

In [9]:
from pymisp import MISPAttribute

attribute = MISPAttribute()
attribute.type = 'domain'
attribute.value = 'circl.lu'

print(attribute.to_json(indent=4))
print(event.add_attribute(**attribute).to_json(indent=4))

{
    "uuid": "4a18fd6c-abed-413d-a386-da32a3e558eb",
    "type": "domain",
    "value": "circl.lu"
}
{
    "uuid": "4a18fd6c-abed-413d-a386-da32a3e558eb",
    "type": "domain",
    "value": "circl.lu",
    "category": "Network activity",
    "to_ids": true,
    "disable_correlation": false
}


In [10]:
# Loaded from a JSON
json = '''{
    "type": "domain",
    "value": "circl.lu",
    "to_ids": false
}'''

json_attribute = MISPAttribute()
json_attribute.from_json(json)

print(json_attribute.to_json(indent=4))

{
    "uuid": "5b6c8fa9-91e2-4aa3-aa78-b22e0114fa6f",
    "type": "domain",
    "value": "circl.lu",
    "category": "Network activity",
    "to_ids": false,
    "disable_correlation": false
}


In [11]:
# Loaded from a python dict
_dict = {
    'type': 'domain',
    'value': 'circl.lu',
    'to_ids': False
}

dict_attribute = MISPAttribute()
dict_attribute.from_dict(**_dict)

print(dict_attribute.to_json(indent=4))

{
    "uuid": "3a64bf51-d153-4696-8e59-7822a8668e1a",
    "type": "domain",
    "value": "circl.lu",
    "category": "Network activity",
    "to_ids": false,
    "disable_correlation": false
}


### Tag an Attribute

The same way to tag events applies for attributes

In [12]:
dict_attribute.add_tag('tlp:clear')

print(dict_attribute.to_json(indent=4))

{
    "uuid": "3a64bf51-d153-4696-8e59-7822a8668e1a",
    "Tag": [
        {
            "name": "tlp:clear"
        }
    ],
    "type": "domain",
    "value": "circl.lu",
    "category": "Network activity",
    "to_ids": false,
    "disable_correlation": false
}


## MISP Object

MISP Objects are containers to group attributes in a way that makes sense. The objects are based on templates that are bundled in the library itself, but you can also use your own templates.



In [13]:
from pymisp import MISPObject

misp_object = MISPObject('domain-ip')
misp_object.comment = 'My fancy new object'

object_attribute = misp_object.add_attribute('domain', value='circl.lu')
object_attribute.add_tag('tlp:green')
misp_object.add_attribute('ip', value='149.13.33.14')
misp_object.add_attribute('first-seen', value='2022-12-31')
misp_object.add_attribute('last-seen', value='2023-01-06')

print(misp_object.to_json(indent=4))

{
    "name": "domain-ip",
    "meta-category": "network",
    "template_uuid": "43b3b146-77eb-4931-b4cc-b66c60f28734",
    "description": "A domain/hostname and IP address seen as a tuple in a specific time frame.",
    "template_version": "11",
    "uuid": "59c852c7-1c10-4d6f-b4f6-e2c86e99a29a",
    "Attribute": [
        {
            "uuid": "e0bba7f7-6ce8-4bb0-a137-e901a33b5726",
            "Tag": [
                {
                    "name": "tlp:green"
                }
            ],
            "object_relation": "domain",
            "value": "circl.lu",
            "type": "domain",
            "category": "Network activity",
            "disable_correlation": false,
            "to_ids": true
        },
        {
            "uuid": "bfbaf6e4-16af-4d60-9bff-ad053fb21e10",
            "object_relation": "ip",
            "value": "149.13.33.14",
            "type": "ip-dst",
            "category": "Network activity",
            "disable_correlation": false,
            

### Short version to add an object to a MISPEvent

You can also add the object directly in a MISP event

In [14]:
from pymisp import MISPObject

misp_object = event.add_object(name='domain-ip', comment='My fancy new object')

object_attribute = misp_object.add_attribute('domain', value='circl.lu')
object_attribute.add_tag('tlp:green')
misp_object.add_attribute('ip', value='149.13.33.14', to_ids=False)
misp_object.add_attribute('first-seen', value='2022-12-31', disable_correlation=False)
misp_object.add_attribute('last-seen', value='2023-01-06')

misp_object.add_reference(attribute1.uuid, 'connects-to')

print(event.to_json(indent=4))

{
    "uuid": "a646652d-10ec-4d31-be98-792d441d77de",
    "Attribute": [
        {
            "uuid": "e45dd98c-7506-4c9e-a22a-4a47c033d2d1",
            "type": "ip-dst",
            "value": "8.8.8.8",
            "category": "Network activity",
            "to_ids": false,
            "disable_correlation": false
        },
        {
            "uuid": "24315449-b8ef-48ec-a4fa-30d7eb5a4dde",
            "type": "ip-dst",
            "value": "8.8.8.9",
            "category": "Network activity",
            "to_ids": false,
            "disable_correlation": true,
            "deleted": true
        },
        {
            "uuid": "30a67450-adfd-4754-86ab-eab73150dee4",
            "type": "ip-src",
            "value": "1.1.1.1",
            "category": "Network activity",
            "to_ids": false,
            "disable_correlation": true
        },
        {
            "uuid": "4a18fd6c-abed-413d-a386-da32a3e558eb",
            "type": "domain",
            "value": "circl.l

# Helpers for MISPObjects

For some objects, we have helpers in order to make your life easier. The most relevant example is the file object: when you have a file to push on MISP, there are plenty of indicators you can extract at once, and it is pretty simple to automate, so we made it a oneliner.

**Note**: This requires a few more dependencies to get the full power of the script: 
* `lief` to extract indicators out of PE/ELF/MachO files, and soon Android binaries.
* `python-magic` to get the mime type
* `pydeep` to compute the ssdeep of the binary whenever possible


```bash
pip install lief python-magic git+https://github.com/kbandla/pydeep.git
```

In [15]:
from pathlib import Path
test_path = Path().resolve().parent / 'PyMISP' / 'tests'
print(test_path)

from pymisp.tools import FileObject

file_object = FileObject(
    filepath=test_path / 'viper-test-files' / 'test_files' / 'EICAR.com',
    standalone=False
)
print(file_object.to_json(indent=4))

/Users/chrisr3d/git/MISP/automation4MISP/PyMISP/tests
{
    "name": "file",
    "meta-category": "file",
    "template_uuid": "688c46fb-5edb-40a3-8273-1af7923e2215",
    "description": "File object describing a file with meta-information",
    "template_version": "25",
    "uuid": "32e7134f-1d70-4d6e-915e-3b62ee1593a8",
    "Attribute": [
        {
            "uuid": "228ea389-4e4b-4e73-ab33-fed662ba4466",
            "object_relation": "filename",
            "value": "EICAR.com",
            "type": "filename",
            "category": "Payload delivery",
            "disable_correlation": true,
            "to_ids": true
        },
        {
            "uuid": "bc0446b6-7a67-45ad-914b-901df52938e9",
            "object_relation": "size-in-bytes",
            "value": "58",
            "type": "size-in-bytes",
            "disable_correlation": true,
            "to_ids": false,
            "category": "Other"
        },
        {
            "uuid": "72f71629-f02c-48f1-bf16-bd5e28b

## Playing with a malware sample

The data you receive out of the JSON dump from a MISP instance is a base64 encoded zip with `infected` as a password.  
The zip file contains 2 files, one containing the original file name of the uploaded file, and the other one is the binary.

This is pretty much a pain to use as-is, so there is an helper for that!

In [16]:
sample = file_object.get_attributes_by_relation('malware-sample')[0]
print(sample)
print('File name --->', sample.malware_filename)
print(sample.malware_binary)
print('Content of the malware (in bytes) ----->', sample.malware_binary.getvalue())

<MISPObjectAttribute(object_relation=malware-sample, value=EICAR.com)
File name ---> EICAR.com
Content of the malware (in bytes) -----> b'X5O!P%@AP[4\\PZX54(P^)7CC)7}$EICAR-STANDARD-ANTIVIRUS-TEST-'


## Use lief to extract indicators out of binaries

Another cool hepler allows you to pass the path of a binary. If the binary's format is supported by `lief`, you get the file object, the binary definition (PE, ELf or Mach-o) and the relevant sections.

If it is anything else, it will simply generate a file object.

In [17]:
from pymisp.tools import make_binary_objects

misp_event = MISPEvent()
misp_event.info = 'Test with binary file'

filepath = test_path / 'viper-test-files' / 'test_files' / 'whoami.exe'
file_obj, bin_obj, sections = make_binary_objects(
    filepath=filepath.as_posix(),
    standalone=False
)

misp_event.add_object(file_obj)
if bin_obj:
    misp_event.add_object(bin_obj)
    for section in sections:
        misp_event.add_object(section)

The references between the different objects are also set by default with the `make_binary_objects` method.

In [18]:
for misp_object in misp_event.objects:
    print(misp_object)

<FileObject(name=file)
<PEObject(name=pe)
<PESectionObject(name=pe-section)
<PESectionObject(name=pe-section)
<PESectionObject(name=pe-section)
<PESectionObject(name=pe-section)
<PESectionObject(name=pe-section)


## CSV support

In [19]:
valid_csv = test_path / 'csv_testfiles' / 'valid_fieldnames.csv'

with open(valid_csv, 'r') as f:
    print(f.read())

md5, sha1, sha256
644087ccca16d2a728ef7685a4106f09, eabd6974ac71efd72d9e0688d5a6131f336d169c, 385e31c97e3a07bbb81513f0cd0979e64e6b014943902efd002f57b21eadd41e
34187a34d0a3c5d63016c26346371b54, ce8209ff9828aa8cb095bd7d1589fc4d394c298c, 5f815b8a8e77731c9ca2b3a07a27f880ef24d54e458d77bdabbbaf2269fe96c3
871aa15f4d61c85e1284e1be3f99f705, 236eac0b19f91117b27f1b198a4d8490d99ec2e5, b434bccf0a5ff75b27184e661df751466aef69f35fbd7b8b8692302b8b886262



In this case, we have valid field names, we can use the file as is and pass it to the CSV loader:

In [20]:
from pymisp.tools import CSVLoader

csv1 = CSVLoader(template_name='file', csv_path=valid_csv)
csv_event = MISPEvent()
csv_event.info = 'Test event from CSV loader'

for o in csv1.load():
    csv_event.add_object(**o)
    
print(csv_event.to_json(indent=4))

{
    "uuid": "bac52e26-0deb-47ef-8d35-a60afc94f9d2",
    "Object": [
        {
            "name": "file",
            "meta-category": "file",
            "template_uuid": "688c46fb-5edb-40a3-8273-1af7923e2215",
            "description": "File object describing a file with meta-information",
            "template_version": "25",
            "uuid": "1b4f6cd8-70d3-407d-b0e3-4bacaa583505",
            "Attribute": [
                {
                    "uuid": "b0fc2dcc-bdc9-425a-95f2-64527637ffcf",
                    "object_relation": "md5",
                    "value": "644087ccca16d2a728ef7685a4106f09",
                    "type": "md5",
                    "disable_correlation": false,
                    "to_ids": true,
                    "category": "Payload delivery"
                },
                {
                    "uuid": "bf7de6c5-a080-486f-b589-c7ddfb286f4e",
                    "object_relation": "sha1",
                    "value": "eabd6974ac71efd72d9e0688d5a6

In [21]:
invalid_csv = test_path / 'csv_testfiles' / 'invalid_fieldnames.csv'

with open(invalid_csv, 'r') as f:
    print(f.read())

SHA1,fileName,size
2a030cc6d84d5785f5e84d0f5888a411d4b06d01,soft.exe,45568
2abae839362edfe52d9ebe282fb61113d22b331f,sttager.exe,20480
6995a32e0a4d4f6d0c9b2a00a96d69bff4b83ea7,test443.exe,373911
87b1f17fbb4a1e8eef4cb31c1c0194b1426c868c,veil.exe,345761
afc36916a4df934446681ea28bef6add4decb98a,80_http.exe.exe,411850
f832d94391a8d2d5cf92773e6c912905ec7c40c7,test1.exe,406636
056823c7891a04b2fec8903eb401ae3291743a54,beca.exe.exe,23808
b7afa7acf1b7ded2c4e3d0884b5cdaa230d9f82e,shell1.exe,24576
4b50b6b9157026ab408d966ece02d1cef8045f82,starggge.exe,27136
6042dfd50d33da40e383baec4a7ef7c75bf17481,8_32.exe,24064



The field names being invalid, we have to specify them with some valid `object_relation` fields.

In our case we also have to tell the CSV loader that we already have field names otherwise the first line is going to be imported in a MISP object.

In [22]:
csv_event = MISPEvent()
csv_event.info = 'Test event from CSV loader'

csv2 = CSVLoader(
    template_name='file',
    csv_path=invalid_csv,
    fieldnames=['sha1', 'filename', 'size-in-bytes'],
    has_fieldnames=True
)

for o in csv2.load():
    csv_event.add_object(**o)
    
print(csv_event.to_json(indent=4))

{
    "uuid": "9dd21dd6-2c01-40f2-b5ca-806ee61cbae0",
    "Object": [
        {
            "name": "file",
            "meta-category": "file",
            "template_uuid": "688c46fb-5edb-40a3-8273-1af7923e2215",
            "description": "File object describing a file with meta-information",
            "template_version": "25",
            "uuid": "2f157803-5054-4daf-9f38-05292eaad85f",
            "Attribute": [
                {
                    "uuid": "26f944db-7427-41c5-b131-7f464a0ef451",
                    "object_relation": "sha1",
                    "value": "2a030cc6d84d5785f5e84d0f5888a411d4b06d01",
                    "type": "sha1",
                    "disable_correlation": false,
                    "to_ids": true,
                    "category": "Payload delivery"
                },
                {
                    "uuid": "135ad98a-16ce-4143-a931-dc1fb0449de0",
                    "object_relation": "filename",
                    "value": "soft.exe",
   

## Generic helper

This helper can be used when you already have a script that does the mapping between your own code and a MISPObject template.

In [24]:
from pymisp.tools import GenericObjectGenerator

attributes_as_dict = [
    {
        'filename': 'shell1.exe',
        'sha1': {
            'value': 'b7afa7acf1b7ded2c4e3d0884b5cdaa230d9f82e',
            'to_ids': False
        },
        'size-in-bytes': {
            'value': 24576,
            'disable_correlation': True
        }
    }
]

misp_object = GenericObjectGenerator('file', strict=True)
misp_object.generate_attributes(attributes_as_dict)

print(misp_object.to_json(indent=4))

{
    "name": "file",
    "meta-category": "file",
    "template_uuid": "688c46fb-5edb-40a3-8273-1af7923e2215",
    "description": "File object describing a file with meta-information",
    "template_version": "25",
    "uuid": "c26d5893-6f56-4287-88b2-1018d10bdd01",
    "Attribute": [
        {
            "uuid": "e120f5ff-76bb-4515-ae41-83245c749604",
            "object_relation": "filename",
            "value": "shell1.exe",
            "type": "filename",
            "category": "Payload delivery",
            "disable_correlation": true,
            "to_ids": true
        },
        {
            "uuid": "f44b3938-b213-49d7-9187-0cc610bef9ab",
            "object_relation": "sha1",
            "value": "b7afa7acf1b7ded2c4e3d0884b5cdaa230d9f82e",
            "type": "sha1",
            "disable_correlation": false,
            "to_ids": false,
            "category": "Payload delivery"
        },
        {
            "uuid": "2ef4d701-b994-4e4c-ae31-24412d5a94ef",
            "

### User defined objects

The Generic helper can also be used to define your own object template.

In [25]:
attributes_as_dict = [
    {
        'MyCoolAttribute': {
            'value': 'critical thing',
            'type': 'text'
        },
        'MyCoolerAttribute': {
            'value': 'even worse',
            'type': 'text'
        }
    }
]

# We cannot use `strict=True` here
misp_object = GenericObjectGenerator('my-cool-template')
misp_object.generate_attributes(attributes_as_dict)

print(misp_object.to_json(indent=4))

{
    "name": "my-cool-template",
    "uuid": "39f84d23-b864-45f2-bc93-cfc72abac3cc",
    "Attribute": [
        {
            "uuid": "4703d884-8bb2-40e9-8058-278079b87803",
            "object_relation": "MyCoolAttribute",
            "value": "critical thing",
            "type": "text",
            "disable_correlation": false,
            "to_ids": false,
            "category": "Other"
        },
        {
            "uuid": "af54288a-148b-4346-b98b-c53cb028ff6c",
            "object_relation": "MyCoolerAttribute",
            "value": "even worse",
            "type": "text",
            "disable_correlation": false,
            "to_ids": false,
            "category": "Other"
        }
    ],
    "distribution": "5",
    "sharing_group_id": "0"
}


PyMISP is OK with this generic object and won't complain if you set the required fields.

Nonetheless, before pushing such event to MISP, we want to set a few additional fields:

In [26]:
from uuid import uuid4

misp_object.template_uuid = uuid4()
misp_object.template_id = 1
misp_object.description = 'foo'
setattr(misp_object, 'meta-category', 'bar')

print(misp_object.to_json(indent=4))

{
    "name": "my-cool-template",
    "uuid": "39f84d23-b864-45f2-bc93-cfc72abac3cc",
    "Attribute": [
        {
            "uuid": "4703d884-8bb2-40e9-8058-278079b87803",
            "object_relation": "MyCoolAttribute",
            "value": "critical thing",
            "type": "text",
            "disable_correlation": false,
            "to_ids": false,
            "category": "Other"
        },
        {
            "uuid": "af54288a-148b-4346-b98b-c53cb028ff6c",
            "object_relation": "MyCoolerAttribute",
            "value": "even worse",
            "type": "text",
            "disable_correlation": false,
            "to_ids": false,
            "category": "Other"
        }
    ],
    "distribution": "5",
    "sharing_group_id": "0",
    "template_uuid": "355e067b-b022-4373-ba61-ac975168ac7b",
    "template_id": "1",
    "description": "foo",
    "meta-category": "bar"
}


### Use locally defined object templates

**Important**: The path you pass as parameter for `misp_objects_path_custom` needs to contain a directory equals to the value of the parameter `name` (same structure as the content of the `misp-object` repository)

In [ ]:
user_defined_obj = MISPObject(
    name='test_object_template',
    strict=True,
    misp_objects_path_custom=test_path / 'mispevent_testfiles'
)

user_defined_obj.add_attribute('member1', 'foo')
user_defined_obj.add_attribute('member2', value='bar', to_ids=True)
user_defined_obj.add_attribute('member3', **{'value': 'baz'})

print(user_defined_obj.to_json(indent=4))